In [3]:
import pandas as pd
import torch
import random
import os
from torch.distributions.binomial import Binomial
from torch.distributions.bernoulli import Bernoulli
import torch.nn as nn

In [ ]:
def treat_portion(x):
    x = x.replace(' [SEP] ', '').strip()
    token_list = x.split(' ')
    treat_cnt = 0
    if len(token_list) > 0:
        for token in token_list:
            if 'diag' not in token:
                treat_cnt += 1
    
    score = (treat_cnt + 1e-8)/ (len(token_list) + 1e-8)
    score = max(score, 0.1)
    score = min(score, 0.9)

    return score

DATA_PATH = '/nfs/turbo/lsa-regier/emr-data/'
groups = [str(i) for i in range(10)]
beta_1 = torch.tensor([1.0, 10.0, 100.0])


for group in groups:
    print(group)
    data_path = os.path.join(DATA_PATH, f'group_{group}_merged.csv')
    data_temp = pd.read_csv(data_path, sep=',')
    
    ### calculate propensity score pi(z)
    data_temp = data_temp.assign(base_propensity_score = data_temp['document'].apply(lambda x: treat_portion(x)))
    
    ### simulate treatment based on propensity score
    all_treatments = Binomial(1, torch.tensor(data_temp["base_propensity_score"].values))
    data_temp = data_temp.assign(treatment = all_treatments.sample())
    
    ### simulate response from treatment and propensity score
    treatment_propensity = 0.25 * data_temp['treatment'] + beta_1[0] * (data_temp['base_propensity_score'] - 0.2)
    sigmoid_fn = torch.nn.Sigmoid()
    input = torch.tensor(treatment_propensity.values)
    reponse_prob = sigmoid_fn(input)
    all_response = torch.bernoulli(reponse_prob)
    data_temp = data_temp.assign(response = all_response)
    data = pd.concat([data, data_temp]) if group != '0' else data_temp

0
1
2
3
4
5
6
7


In [ ]:
data["response"].sum()

In [ ]:
data["treatment"].sum()

In [ ]:
data
